# <01_데이터 전처리>

* 데이터 전처리
* EDA
* 분석 내용
* 인사이트

In [1]:
import pandas as pd
import openpyxl

In [2]:
data = pd.read_excel('data/data.xlsx')
data.head()

,날짜,메뉴,포스번호,영수번호,시간,수량,매출,메뉴번호,요일,날씨
0,20221125,[40863]캬라멜팝콘(M),2,73,1956,1,5000,40863,금,비
1,20221030,[40860]컵와인 까베르네소비뇽,1,293,1912,2,18182,40860,일,맑음
2,20221109,[00413]아이스 아메리카노(Ice Coffee),1,43,1931,1,4545,413,수,흐림
3,20221019,[40863]캬라멜팝콘(M),1,20,1925,1,5000,40863,수,맑음
4,20221112,[15867]츄러스,1,109,1328,1,4545,15867,토,비


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      40000 non-null  int64 
 1   메뉴      40000 non-null  object
 2   포스번호    40000 non-null  int64 
 3   영수번호    40000 non-null  int64 
 4   시간      40000 non-null  int64 
 5   수량      40000 non-null  int64 
 6   매출      40000 non-null  int64 
 7   메뉴번호    40000 non-null  int64 
 8   요일      40000 non-null  object
 9   날씨      40000 non-null  object
dtypes: int64(7), object(3)
memory usage: 3.1+ MB


In [4]:
data['날짜'].min(), data['날짜'].max()

(20221019, 20230101)

# 1.데이터 전처리

## 1-1.메뉴번호와 메뉴명 분리

In [5]:
data['메뉴'] = data['메뉴'].str.split(']').str[1]

## 1-2.앞뒤 공백 제거
* ex) '코카콜라 ', '스프라이트 ', ...

In [6]:
data['메뉴'] = data['메뉴'].str.strip()

## 1-3.날짜+시간

In [7]:
data['일시'] = pd.to_datetime(data['날짜'].astype(str) + (data['시간'] * 100).astype(str))
data2 = data.drop(columns=['날짜', '시간'], axis=1)

In [8]:
data2 = data2[['일시', '요일', '날씨',
               '포스번호', '영수번호', '메뉴번호', '메뉴',
               '수량', '매출']]
data2.head()

,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출
0,2022-11-25 19:56:00,금,비,2,73,40863,캬라멜팝콘(M),1,5000
1,2022-10-30 19:12:00,일,맑음,1,293,40860,컵와인 까베르네소비뇽,2,18182
2,2022-11-09 19:31:00,수,흐림,1,43,413,아이스 아메리카노(Ice Coffee),1,4545
3,2022-10-19 19:25:00,수,맑음,1,20,40863,캬라멜팝콘(M),1,5000
4,2022-11-12 13:28:00,토,비,1,109,15867,츄러스,1,4545


## 1-4.메뉴 정리
* 소떡 소떡 제거
    - 실제 메뉴판 이미지에서 발견되지 않음.
* 쵸코라떼 (HOT)와 핫초코는 같은 메뉴로 판단되어 핫초코로 통합
    - 판매일자가 겹치지 않음.
    - 매출액이 4545로 같음.
    - 쵸코라떼 (HOT) 19개 < 핫초코 606개

In [11]:
data2['메뉴'].unique()

array(['캬라멜팝콘(M)', '컵와인 까베르네소비뇽', '아이스 아메리카노(Ice Coffee)', '츄러스',
       '컵와인 샤도네이', '생수', '코카콜라', '캬라멜 팝콘(L)', 'Couple set A', '뉴욕핫도그',
       '컵와인 핑크모스카토', '아이스티(복숭아)', '클라우드(캔)', '토레타', '스프라이트',
       '아메리카노(Americano)', '미닛메이드 포도', '순살닭강정', '핫초코', '오리지널 팝콘(M)',
       'Single set', '프레즐', '(배민)양념 소떡소떡(1p)', 'Beer set', 'Family set',
       '허브티', '사케', '카페라떼', '미닛메이드 오렌지', '하겐다즈 바닐라', '하겐다즈 쵸코',
       '오리지널 팝콘(L)', '하겐다즈 딸기', '쵸코라떼 (HOT)', '뱅쇼(G)', 'Couple set B',
       '샤퀴테리 Box (4구)'], dtype=object)

In [12]:
data3 = data2.drop(data2.loc[data2['메뉴'] == '(배민)양념 소떡소떡(1p)'].index)

In [15]:
display(data3.loc[data['메뉴'] == '쵸코라떼 (HOT)'].head(1))
display(data3.loc[data['메뉴'] == '핫초코'].head(1))

,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출
1611,2022-10-23 17:14:00,일,맑음,1,179,40855,쵸코라떼 (HOT),1,4545


,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출
84,2022-12-29 20:21:00,목,맑음,3,138,448,핫초코,1,4545


In [17]:
data3.loc[data3['메뉴번호'] == 40855, '메뉴번호'] = 448
data3.loc[data3['메뉴'] == '쵸코라떼 (HOT)', '메뉴'] = '핫초코'

## 1-5.매출액 정리
* 수량에 따른 매출액이 조금씩 차이나는 데이터 정리

In [18]:
data_price = data3.loc[:, ['메뉴번호', '메뉴', '수량', '매출']]
data_price = data_price.drop_duplicates().sort_values(by=['메뉴번호', '수량'], ignore_index=True)
data_price

,메뉴번호,메뉴,수량,매출
0,410,아메리카노(Americano),1,4091
1,410,아메리카노(Americano),2,8182
2,410,아메리카노(Americano),3,12273
3,410,아메리카노(Americano),4,16364
4,410,아메리카노(Americano),5,20455
...,...,...,...,...
193,40879,Beer set,1,16364
194,40879,Beer set,2,32727
195,40879,Beer set,3,49091
196,40879,Beer set,4,65455


In [19]:
data_price2 = data_price.loc[data_price['수량'] == 1].drop(columns='수량', axis=1).reset_index(drop=True)
data_price2.rename(columns={'매출':'가격'}, inplace=True)
data_price2.head()

,메뉴번호,메뉴,가격
0,410,아메리카노(Americano),4091
1,413,아이스 아메리카노(Ice Coffee),4545
2,422,카페라떼,4545
3,448,핫초코,4545
4,2336,사케,10000


In [20]:
data_price2.to_csv('data/price.csv', index=False, encoding='cp949')

In [25]:
# pd.read_csv('data/price.csv', encoding='cp949')
data_m = pd.merge(data3, data_price2.drop(columns='메뉴', axis=1), how='inner', on='메뉴번호')
data_m['매출'] = data_m['가격'] * data_m['수량']

In [26]:
data4 = data_m.drop(columns='가격', axis=1).sort_values(by=['일시', '영수번호'], ignore_index=True)
data4.head()

,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출
0,2022-10-19 19:01:00,수,맑음,3,1,14097,생수,1,909
1,2022-10-19 19:01:00,수,맑음,3,1,32487,아이스티(복숭아),1,4545
2,2022-10-19 19:03:00,수,맑음,1,1,32292,스프라이트,1,3182
3,2022-10-19 19:04:00,수,맑음,5,1,413,아이스 아메리카노(Ice Coffee),1,4545
4,2022-10-19 19:04:00,수,맑음,5,1,14097,생수,1,909


In [27]:
# data_preprocessed
data4.to_csv('data/data_pre.csv', index=False, encoding='cp949')

# 2.데이터 생성

In [28]:
# pd.options.display.max_columns = 20
# pd.options.display.max_rows = 60
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [33]:
df = pd.read_csv('data/data_pre.csv', encoding='cp949')
df['일시'] = pd.to_datetime(df['일시'])
df.head()

,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출
0,2022-10-19 19:01:00,수,맑음,3,1,14097,생수,1,909
1,2022-10-19 19:01:00,수,맑음,3,1,32487,아이스티(복숭아),1,4545
2,2022-10-19 19:03:00,수,맑음,1,1,32292,스프라이트,1,3182
3,2022-10-19 19:04:00,수,맑음,5,1,413,아이스 아메리카노(Ice Coffee),1,4545
4,2022-10-19 19:04:00,수,맑음,5,1,14097,생수,1,909


## 2-1.고객별 구매 데이터
* 포스번호와 영수번호가 같은 데이터들은 한 고객의 구매 데이터로 정의

In [34]:
# 일시, 포스번호, 영수번호로 새 Feature 생성
df2 = df.copy()
df2['구매번호'] = df2['일시'].dt.month.astype(str) + df2['일시'].dt.day.astype(str) + \
                '-' + df2['포스번호'].astype(str) + '-' + df2['영수번호'].astype(str)

In [35]:
# 수량이 N(N>1)인 데이터를 수량 1인 데이터 N개로 복제
df_expand = pd.DataFrame()

for index, row in df2.iterrows():
    count = row['수량']
    for _ in range(count-1):
        df_expand = df_expand.append(row, ignore_index=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_6776\2101654794.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_expand = df_expand.append(row, ignore_index=True)


In [40]:
# 기존 데이터와 복제 데이터 병합
df3 = pd.concat([df2, df_expand], join='inner', axis=0)
df3.loc[df3['수량'] > 1, '수량'] = 1
df3.sort_values(by=['일시', '메뉴번호'], ignore_index=True, inplace=True)
df3.loc[12:21]

,일시,요일,날씨,포스번호,영수번호,메뉴번호,메뉴,수량,매출,구매번호
12,2022-10-19 19:07:00,수,맑음,2,2,15867,츄러스,1,4545,1019-2-2
13,2022-10-19 19:07:00,수,맑음,2,2,32291,코카콜라,1,3182,1019-2-2
14,2022-10-19 19:07:00,수,맑음,2,2,34433,순살닭강정,1,14546,1019-2-2
15,2022-10-19 19:07:00,수,맑음,2,2,34433,순살닭강정,1,14546,1019-2-2
16,2022-10-19 19:07:00,수,맑음,2,2,40858,컵와인 샤도네이,1,9091,1019-2-2
17,2022-10-19 19:07:00,수,맑음,2,2,40859,컵와인 핑크모스카토,1,9091,1019-2-2
18,2022-10-19 19:07:00,수,맑음,2,2,40860,컵와인 까베르네소비뇽,1,18182,1019-2-2
19,2022-10-19 19:07:00,수,맑음,2,2,40860,컵와인 까베르네소비뇽,1,18182,1019-2-2
20,2022-10-19 19:08:00,수,맑음,3,4,15867,츄러스,1,4545,1019-3-4
21,2022-10-19 19:08:00,수,맑음,3,4,40861,오리지널 팝콘(M),1,5000,1019-3-4


In [41]:
# data_preprocessed_expanded
df3.to_csv('data/data_pre_exp.csv', index=False, encoding='cp949')

In [42]:
customer = df3.pivot_table(index='구매번호', columns='메뉴', aggfunc='size').fillna(0).astype(int)
customer.head(2)

메뉴,Beer set,Couple set A,Couple set B,Family set,Single set,뉴욕핫도그,미닛메이드 오렌지,미닛메이드 포도,뱅쇼(G),사케,생수,샤퀴테리 Box (4구),순살닭강정,스프라이트,아메리카노(Americano),아이스 아메리카노(Ice Coffee),아이스티(복숭아),오리지널 팝콘(L),오리지널 팝콘(M),츄러스,카페라떼,캬라멜 팝콘(L),캬라멜팝콘(M),컵와인 까베르네소비뇽,컵와인 샤도네이,컵와인 핑크모스카토,코카콜라,클라우드(캔),토레타,프레즐,하겐다즈 딸기,하겐다즈 바닐라,하겐다즈 쵸코,핫초코,허브티
구매번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1019-1-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1019-1-10,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
customer.to_csv('data/customer.csv')